# Problem Statement

When testing complex REDCap scripts or functionalities, it is **essential** that one tests them in the Test project first before running/using them on the target project.

# Solution

This notebook exports data from the target project, then import them into the test project. The test project then will have the exact same records as the target project, effectively making them the same.

### **This notebook will wipe out all existing data from the Test project before the import. The existing data will still be recoverable through a JSON file.** 

# Instructions
Currently, the API does not allow for importing instruments. Importing instruments has to be done manually.

1. Download all instrument zips from target project.
2. Import the instrument zips into test project.
- Delete current instruments from test project if needed. It is recommended to do this. 
- Before deleting current instruments, download zip files of these instruments.
3. Configure repeating instruments if needed.
4. Run this notebook.



# Code 
### 1. Setup target and test project

In [11]:
import AMBRA_Backups
import json
import pandas as pd
from datetime import datetime

In [2]:
# TARGET PROJECT
target_project_name = 'I3C DECADE - Image Tracking'
target_project = AMBRA_Backups.redcap_funcs.get_redcap_project(target_project_name)

# TEST PROJECT
test_project_name = 'Personal Test'
test_project = AMBRA_Backups.redcap_funcs.get_redcap_project(test_project_name)

Wipe out all existing data from `test_project`.

In [ ]:
record_column = 'record_id'
all_records = test_project.export_records(format_type='df')
all_records.reset_index(inplace=True)

# Get record IDs from Test
all_records_ids = []
if record_column in all_records:
   all_records_ids = all_records['record_id'].unique()

# Before deleting records, export all current records into a json file
all_records_json = test_project.export_records(export_blank_for_gray_form_status=True)
json_obj = json.dumps(all_records_json)

with open(f'{test_project_name}_{datetime.now()}.json', 'w') as f:
   f.write(json_obj)


In [ ]:
# Delete all records
if len(all_records_ids) > 0:
   test_project.delete_records(records=all_records_ids)

test_current_records = test_project.export_records(format_type='df')

if len(test_current_records) == 0:
   print(f'All records from {test_project_name} REDCap have been deleted')
else:
   raise ValueError(f'''Some records remain in {test_project_name} REDCap. current records:
         ---------
         {test_current_records}
         ''')

All records from <redcap.project.Project object at 0x105e4eff0> REDCap have been deleted


### 2. Copy records

In [ ]:
'''
Get current fields from test_project to use as fields filter for
target_project's export records. 

This is needed, as when fields get deleted from a project, they 
will still show in the export_records() if there are values
in them. 

Since test_project does not have data yet, it would have only the
relevant fields and not the deleted fields.
'''
test_fields_dict = test_project.export_field_names()
test_fields = set()

for field in test_fields_dict:
   test_fields.add(field['original_field_name'])

target_records = target_project.export_records(
   fields=test_fields, 
   export_blank_for_gray_form_status=True
)
target_records

[{'record_id': 'B00000610',
  'id': 'B00000610',
  'patient_status_1': '',
  'site': '2',
  'name': '',
  'dob': '',
  'mrn': '',
  'scan_date': '2023-07-28 12:00',
  'in_epic': '',
  'created': '',
  'epic_order_status': '',
  'scheduling_email': '',
  'metallic_implants': '',
  'specify': '',
  'technologist_confirmation': '',
  'epic_upload': '',
  'outlook': '',
  'scheduling_complete': '2',
  'sequences_collected___1': '1',
  'sequences_collected___2': '1',
  'sequences_collected___3': '1',
  'sequences_collected___0': '0',
  'sequences_collected___ni': '0',
  'withdraw_from_imaging': '',
  'please_explain': '',
  'eprime_upload': '2',
  'physio_upload': '3',
  'report_upload': '1',
  'image_qaqc': '1',
  'routed_to_namespace': '1',
  'central_read_status': '1',
  'please_explain_1': '',
  'crf_verification_status': '1',
  'please_explain_2': '',
  'query_status': '',
  'query_description': '',
  'notes': '',
  'inteleshare_processing_complete': '2',
  'incidental_findings_reporte

Define fields that need to be taken off. 

This is necessary in cases where options were defined but got deleted from a variable. They would still retain in `export_records` if there's data in them. This is a manual process. 

Instructions:

1. Set `delete_fields = []`. Run cell.
2. If an error is thrown, copy the fields mention in the error into 
`delete_fields`.
3. Run cell again.

In [7]:
delete_fields = [
   'sequences_acquired___ni',
   'sequences_collected___ni'
]

for record in target_records:
   for field in delete_fields:
      record.pop(field, None)

Import data into test_project

In [ ]:
test_project.import_records(target_records)

TypeError: exceptions must derive from BaseException

In [ ]:
# See if records imported
test_records_df = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)
target_records_df = target_project.export_records(format_type='df', export_blank_for_gray_form_status=True, fields=test_fields)

merged_df = pd.merge(test_records_df, target_records_df, how='outer', indicator=True)
merged_df.to_excel(f'{test_project_name}_{target_project_name}_diff_{datetime.now().xlsx}')

ValueError: You are trying to merge on float64 and object columns for key 'icf_upload'. If you wish to proceed you should use pd.concat